In [7]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os, random, glob, ntpath

import torch, torchaudio
import torchaudio.functional as F
from IPython.display import Audio

mode = "train"

speech,_ = torchaudio.load("data/speech/train/test.wav")
print(speech.shape)
noise,_ = torchaudio.load("data/noise/train/ch01.wav")
print(noise.shape)
noise = noise[:,: speech.shape[1]]

snr_dbs = torch.tensor([5])
noisy_speech = F.add_noise(speech,noise, snr_dbs)
Audio(noisy_speech, rate=16000)

speech_list = glob.glob("data/speech/" + mode + "/*.wav")
noise_list = glob.glob("data/noise/" + mode + "/*.wav")

for speech_file in speech_list:
        snr = random.randint(-1,2) * 5
        noise_file = random.choice(noise_list)

        speech_name = ntpath.basename(speech_file).removesuffix(".wav")
        noise_name = ntpath.basename(noise_file).removesuffix(".wav")
        output_path = f"{"data/mixed/"+mode}/{speech_name}_{noise_name}_{snr}.wav"